In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# **Gemini Enterprise custom agent with prompt management**

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/search/gemini-enterprise/gemini_enterprise_prompt_management.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fsearch%2Fgemini-enterprise%2Fgemini_enterprise_prompt_management.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/search/gemini-enterprise/gemini_enterprise_prompt_management.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/gemini-enterprise/gemini_enterprise_prompt_management.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>



<br>
<br>
<br>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/gemini-enterprise/gemini_enterprise_prompt_management.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/gemini-enterprise/gemini_enterprise_prompt_management.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/gemini-enterprise/gemini_enterprise_prompt_management.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/gemini-enterprise/gemini_enterprise_prompt_management.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/gemini-enterprise/gemini_enterprise_prompt_management.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Authors |
| --- |
| Rupjit Chakraborty|
| Sharmila Devi |
| Parag Mhatre |

## Overview : Custom agent with file context and prompt management in Gemini Enterprise

This notebook demonstrates an end-to-end example of a custom agent integrated with Gemini Enterprise that can accept a file as context and generate a response using that. In this case, a simple example of a SQL query generator based on a given schema DDL has been demonstrated. We also demonstrate custom agent [prompt management using Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/prompt-classes)

### Background

Custom agents integrated with Gemini Enterprise by developers may require instruction modification based on the user choice. Also, the instructions given to agents via prompts, need to be maintainable via versioning. The prompt management sdk by Vertex AI can be used to manage prompts for agents at scale.

### Business Application Scenario

We demonstrate a simple scenario where a prompt is acquired by a custom agent deployed on Agent Engine via Vertex AI Prompt Management sdk. The end-user or business users can update the prompt whenever needed and the agent can just be redeployed to read the new prompt, without any changes in the code.

### Notebook Overview
This notebook guides you through the following key steps:

**1. Creation of prompt using Vertex AI Prompt Management**  
&nbsp;&nbsp;&nbsp;&nbsp;a. Create the prompt and get the prompt id

**2. Creation of custom agent that uses the prompt in Vertex AI Prompt Management**  
&nbsp;&nbsp;&nbsp;&nbsp;a. Create the custom agent using the prompt id  
&nbsp;&nbsp;&nbsp;&nbsp;b. Test and Deploy the agent to Agent Engine

**3. Integration of custom agent to Gemini Enterprise**  
&nbsp;&nbsp;&nbsp;&nbsp;a. Register the agent with Gemini Enterprise and integrate it

**4. Demonstration of prompt change for custom agent without re-deploying custom agent**  
&nbsp;&nbsp;&nbsp;&nbsp;a. Make a change in prompt being used by the agent and demonstrate the change in output

## Imports

In [ ]:
import vertexai
from google.adk.agents import Agent
from google.adk.agents.callback_context import CallbackContext
from google.genai import types
from vertexai import agent_engines
from vertexai.preview import prompts, reasoning_engines
from vertexai.preview.prompts import Prompt

## Constants and Initializations

In [ ]:
# TODO for Developer: Update project name, location, bucket and schema path (for testing)
PROJECT_ID = "[your-project-id]"
PROJECT_NUMBER = "[your-project-number]"
LOCATION = "[your-location]"
STAGING_BUCKET = "[your-bucket]"
SCHEMA = "[your-schema-txt-containing-DDLs]"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

try:
    with open(SCHEMA) as f_handle:
        schema = f_handle.read()
except FileNotFoundError:
    print(f"Error: The schema file was not found at '{SCHEMA}'.")
    print("Please update the 'SCHEMA' variable with the correct file path.")
    schema = ""  # Assign a default value to prevent further errors

## Prompt creation using Vertex AI Prompt Management sdk

In [ ]:
# Add a new prompt to the Prompt Management store via sdk
# This can also be done via the UI
# This task needs to be done only once

sql_query_gen_prompt = """
Look carefully at the schema DDL statements and provide the SQL query for the user's question.
Also provide a detailed explanation of the query generated.
If you cannot find a proper schema return "Please upload a schema or a valid schema".
Do not assume anything.
"""

sql_query_gen_prompt_obj = Prompt(
    prompt_name="SQL Generator",
    prompt_data=sql_query_gen_prompt,
    model_name="gemini-2.0-flash-001",
)
agent_prompt = prompts.create_version(prompt=sql_query_gen_prompt_obj)
print("Prompt id = ", agent_prompt.prompt_id)

![SQL generation prompt in Prompt Management"](https://services.google.com/fh/files/blogs/prompt_mgmt.png)

In [ ]:
# List all the prompts and copy the prompt_id of the recently created prompt
# Alternatively we can get is as: pid = agent_prompt.prompt_id

prompts.list()

## Create Agent

In [ ]:
"""SQL query generator agent"""

# prompt id obtained after listing prompts
PID = "[your-prompt-id]"


def update_instructions_add_schema(callback_context: CallbackContext):
    """Add the contents of the file to the agent context.

    Args:
        callback_context: The agent context.
    """
    from vertexai.preview import prompts

    flag = 0
    # acquire the prompt added by an end user and add based instructions to it
    sql_query_gen_prompt = prompts.get(prompt_id=PID).prompt_data
    # add the prompt instruction to the agent context
    callback_context._invocation_context.agent.instruction = sql_query_gen_prompt

    # iterate over the callback_context object and check if file data is present
    for part in callback_context.user_content.parts:
        if hasattr(part, "inline_data") and getattr(part.inline_data, "mime_type", ""):
            blob = part.inline_data
            raw_bytes = blob.data
            flag = 1

    # update the context if there is any file data found
    if flag and raw_bytes and isinstance(raw_bytes, (bytes, bytearray)):
        uploaded_file_content = raw_bytes.decode("utf-8")
        callback_context._invocation_context.agent.instruction += uploaded_file_content


# root agent that does the SQL generation
root_agent = Agent(
    name="SQL_Generation_Agent",
    model="gemini-2.0-flash",
    description=(
        "Agent to convert natural language query to SQL based on a given schema DDL."
    ),
    before_agent_callback=update_instructions_add_schema,
    generate_content_config=types.GenerateContentConfig(temperature=0.01),
)

## Test agent locally

In [ ]:
# create an AdkApp object for testing and deployment of agent

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [ ]:
# create a session for interacting with agent

session = app.create_session(user_id="u_123")
session

In [ ]:
# build the user query object for getting response from agent

query = "list all benefits of every employee mapped to the employee name"

file_artifact = types.Part.from_bytes(
    mime_type="text/plain", data=schema.encode("utf-8")
)
contents = types.Content(
    role="user", parts=[types.Part.from_text(text=query), file_artifact]
)
contents.model_dump()

In [ ]:
# get output for the user query

for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message=contents.model_dump(),
):
    print(event["content"]["parts"][0]["text"])

## Deploy agent to Agent Engine

In [ ]:
# add the necessary dependencies and install agent to agent engine

remote_app = agent_engines.create(
    display_name="SQL generator",
    agent_engine=app,
    requirements=[
        "google-adk (==1.5.0)",
        "google-genai (==1.24.0)",
        "pydantic (==2.11.7)",
        "google-cloud-aiplatform (==1.101.0)",
    ],
)

In [ ]:
# get the reasoning engine id

remote_app.resource_name

## Register agent in Gemini Enterprise

In [ ]:
%%bash

curl -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: [your-project-id]" \
"https://discoveryengine.googleapis.com/v1alpha/projects/[your-project-id]/locations/global/authorizations?authorizationId=sqlgen2" \
-d '{
"name": "projects/[your-project-id]/locations/global/authorizations/sqlgen2",
"serverSideOauth2": {
"clientId": "[UPDATE-CLIENT-ID]",
"clientSecret": "[UPDATE-CLIENT-SECRET]",
"authorizationUri": "https://accounts.google.com/o/oauth2/v2/auth?client_id=[UPDATE-CLIENT-ID]&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&include_granted_scopes=true&response_type=code&access_type=offline&prompt=consent",
"tokenUri": "https://oauth2.googleapis.com/token"
}
}'

In [ ]:
%%bash

curl -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: [your-project-id]" \
"https://discoveryengine.googleapis.com/v1alpha/projects/[your-project-id]/locations/global/collections/default_collection/engines/[your-gemini-enterprise-engine-id]/assistants/default_assistant/agents" \
-d '{
"displayName": "DDL SQL Generator",
"description": "SQL generation agent to help user with SQL queries for a give schema.",
"adk_agent_definition": {
"tool_settings": {
"tool_description": "You are an expert SQL developer capable of understanding user queries and converting them to SQL"
},
"provisioned_reasoning_engine": {
"reasoning_engine": "projects/[your-project-number]/locations/us-central1/reasoningEngines/[reasoning-engine-id]"
},
"authorizations": ["projects/[your-project-number]/locations/global/authorizations/sqlgen2"]
}
}'

## SQL Generator agent in Gemini Enterprise

![DDL based SQL query generator agent](https://services.google.com/fh/files/blogs/sql_gen.png)